In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

2025-04-24 10:13:31.089943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745489611.350824      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745489611.427458      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load the CSV file
data_path = '/kaggle/input/ham10000-preprocessed-dataset/balanced and standardized images meta for custom.csv'  # Change this to your CSV file path
image_folder = '/kaggle/input/ham10000-preprocessed-dataset/balanced_and_standardized_images/balanced_and_standardized_images'  # Change this to your image folder path
df = pd.read_csv(data_path)

# Define parameters
IMG_SIZE = 96
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.5
EPOCHS = 20

# Get unique classes and create label mapping
classes = df['dx'].unique()
class_to_idx = {c: i for i, c in enumerate(classes)}
df['label'] = df['dx'].map(class_to_idx)

# Extract labels and encode them
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["dx"])  # Encode the 'dx' column

In [3]:
# Train-test split8
train_df, test_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)

In [4]:
# Image Data Generators with Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of train set will be used as validation
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

Found 26284 validated image filenames belonging to 7 classes.
Found 6570 validated image filenames belonging to 7 classes.
Found 14081 validated image filenames belonging to 7 classes.


In [5]:
# Load EfficientNetB0 model
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze the base model

# Add custom top layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(DROPOUT_RATE)(x)
output = Dense(len(label_encoder.classes_), activation="softmax")(x)

# Compile the model
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss="categorical_crossentropy", metrics=["accuracy"])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

2025-04-24 10:17:03.415679: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    #callbacks=[early_stop]
)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


822/822 ━━━━━━━━━━━━━━━━━━━━ 620s 727ms/step - accuracy: 0.1428 - loss: 1.9808 - val_accuracy: 0.1438 - val_loss: 1.9459
Epoch 2/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 359s 435ms/step - accuracy: 0.1422 - loss: 1.9487 - val_accuracy: 0.1405 - val_loss: 1.9460
Epoch 3/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 358s 433ms/step - accuracy: 0.1413 - loss: 1.9468 - val_accuracy: 0.1405 - val_loss: 1.9460
Epoch 4/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 381s 461ms/step - accuracy: 0.1446 - loss: 1.9469 - val_accuracy: 0.1405 - val_loss: 1.9460
Epoch 5/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 384s 465ms/step - accuracy: 0.1407 - loss: 1.9467 - val_accuracy: 0.1390 - val_loss: 1.9460
Epoch 6/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 399s 483ms/step - accuracy: 0.1423 - loss: 1.9465 - val_accuracy: 0.1405 - val_loss: 1.9460
Epoch 7/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 363s 439ms/step - accuracy: 0.1413 - loss: 1.9465 - val_accuracy: 0.1405 - val_loss: 1.9460
Epoch 8/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 357s 433ms/step - accuracy: 0.1434 - loss: 1.94

In [7]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], 
            name='train_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], 
            name='val_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], 
            name='train_loss'), 
        row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], 
            name='val_loss'), 
        row=1, col=2)
    fig.show()

In [1]:
plot_model_training_curve(history)

NameError: name 'plot_model_training_curve' is not defined

In [9]:
# Test function
def evaluate_model():
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'\nTest Accuracy: {test_acc * 100:.2f}%')

# Run evaluation
evaluate_model()

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.



441/441 ━━━━━━━━━━━━━━━━━━━━ 239s 542ms/step - accuracy: 0.1412 - loss: 1.9459

Test Accuracy: 14.29%


In [10]:
model.save('/kaggle/working/effecintnet 80-20.keras') 

In [11]:
model.save_weights('/kaggle/working/effecientnet 80-20.weights.h5') 